In [2]:
!git clone https://github.com/google/model_search.git

Cloning into 'model_search'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 203 (delta 6), reused 22 (delta 4), pack-reused 170
Receiving objects: 100% (203/203), 252.61 KiB | 9.02 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
%cd model_search
!pwd

/content/model_search
/content/model_search


In [4]:
!ls

CONTRIBUTING.md  LICENSE  model_search	README.md  requirements.txt  WORKSPACE


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 516.2MB 35kB/s 
     |████████████████████████████████| 133kB 54.1MB/s 
     |████████████████████████████████| 358kB 51.3MB/s 
     |████████████████████████████████| 2.8MB 50.5MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 460kB 51.1MB/s 
     |████████████████████████████████| 3.0MB 47.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=b27ae4948a4b3869c54632dd8f25a1f4e196726343a2e976bedbc2fada60733b
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=eddf4583594c7a0587b3522bf099baa6f8e4f4e85c04c203a30a3058b8e17664
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
  Found existing inst

In [6]:
!protoc --python_out=./ model_search/proto/phoenix_spec.proto
!protoc --python_out=./ model_search/proto/hparam.proto
!protoc --python_out=./ model_search/proto/distillation_spec.proto
!protoc --python_out=./ model_search/proto/ensembling_spec.proto
!protoc --python_out=./ model_search/proto/transfer_learning_spec.proto


In [7]:
# change data path to conatin the processed data
data_path = "/content/drive/MyDrive/model_search/data/"
!ls "/content/drive/MyDrive/model_search/data/"

Cell1.expr.csv	Cell1.train.csv  Cell2.expr.csv  Cell2.train.csv  ProcessedData
Cell1.test.csv	Cell1.valid.csv  Cell2.test.csv  Cell2.valid.csv


In [8]:
# Load and combine data
import pandas as pd
import numpy as np

cell1_expr = pd.read_csv(data_path+"Cell1.expr.csv",header=None)
cell2_expr = pd.read_csv(data_path+"Cell2.expr.csv",header=None)
cell1_train = pd.read_csv(data_path+"Cell1.train.csv", header=None)
cell2_train = pd.read_csv(data_path+"Cell2.train.csv", header=None)

diff = cell1_expr
diff[1] = np.log(cell1_expr[1]+1) - np.log(cell2_expr[1]+1)
diff_dict = diff.set_index(0).T.to_dict('list')

# print(cell1_expr[cell1_expr[0]=="ENSG00000130749"])
# print(cell2_expr[cell2_expr[0]=="ENSG00000130749"])
# print(cell1_train[cell1_train[0].str.contains("ENSG00000130749")])
# print(cell2_train[cell2_train[0].str.contains("ENSG00000130749")])

data = [cell1_train[0], cell1_train[1],cell1_train[2],cell1_train[3],cell1_train[4],cell1_train[5],cell2_train[1],cell2_train[2],cell2_train[3],cell2_train[4],cell2_train[5]]
combined = pd.concat(data,axis=1)
combined[combined.columns[0]] = combined[combined.columns[0]].str.extract(r'(^[^_]+)')
combined[0] = combined[0].replace(diff_dict)

In [9]:
# Label data
lower = -3
upper = 2

copy = combined.copy(True)
copy.loc[(combined[0] > lower)&(combined[0] <= upper),0]=1
copy.loc[(combined[0] > upper),0]=2
copy.loc[(combined[0] <= lower),0]=0
copy = copy.astype(int)

print(copy[copy[0]==0].size)
print(copy[copy[0]==1].size)
print(copy[copy[0]==2].size)

66000
358600
15400


In [10]:
copy.to_csv("tmp.csv",header=None,index=False)

In [11]:
# Use google model search to create trainer
import model_search
from model_search import constants
from model_search import single_trainer
from model_search.data import csv_data

trainer = single_trainer.SingleTrainer(
    data=csv_data.Provider(
        label_index=0,
        logits_dimension=2,
        record_defaults=[0,0,0,0,0,0,0,0,0,0,0],
        filename="tmp.csv"),
    spec='model_search/configs/dnn_config.pbtxt')

In [12]:
!mkdir /tmp/run_example

In [13]:
!rm -r /tmp/run_example/

In [14]:
import sys
from absl import app
sys.argv = sys.argv[:1]

try:
  app.run(lambda argv: None)
except:
  pass

In [15]:
# Use google model search with any number of trials
trainer.try_models(
    number_models=200,
    train_steps=5,
    eval_steps=1,
    root_dir="/tmp/run_example",
    batch_size=512,
    experiment_name="example",
    experiment_owner="model_search_user")

I0416 20:53:19.649744 140341206681472 metadata_store.py:93] MetadataStore with DB connection initialized
I0416 20:53:19.665766 140341206681472 oss_trainer_lib.py:290] creating directory: /tmp/run_example/tuner-1/1
I0416 20:53:19.667040 140341206681472 oss_trainer_lib.py:337] Tuner id: tuner-1
I0416 20:53:19.668069 140341206681472 oss_trainer_lib.py:338] Training with the following hyperparameters: 
I0416 20:53:19.669157 140341206681472 oss_trainer_lib.py:339] {'learning_rate': 6.045962827343877e-05, 'new_block_type': 'FULLY_CONNECTED_PYRAMID', 'optimizer': 'rmsprop', 'initial_architecture_0': 'FULLY_CONNECTED_RESIDUAL_FORCE_MATCH_SHAPES_BATCHNORM', 'exponential_decay_rate': 0.8645748216745901, 'exponential_decay_steps': 2, 'gradient_max_norm': 3, 'dropout_rate': 0.3000000104308128, 'initial_architecture': ['FULLY_CONNECTED_RESIDUAL_FORCE_MATCH_SHAPES_BATCHNORM']}


INFO:tensorflow:TF_CONFIG environment variable: {'model_dir': '/tmp/run_example/tuner-1/1', 'session_master': ''}


I0416 20:53:19.670751 140341206681472 run_config.py:550] TF_CONFIG environment variable: {'model_dir': '/tmp/run_example/tuner-1/1', 'session_master': ''}


INFO:tensorflow:Using model_dir in TF_CONFIG: /tmp/run_example/tuner-1/1


I0416 20:53:19.673529 140341206681472 run_config.py:973] Using model_dir in TF_CONFIG: /tmp/run_example/tuner-1/1


...

INFO:tensorflow:Saving checkpoints for 0 into /tmp/run_example/tuner-1/22/model.ckpt.


I0416 21:33:06.317706 140341206681472 basic_session_run_hooks.py:618] Saving checkpoints for 0 into /tmp/run_example/tuner-1/22/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


I0416 21:33:06.473377 140341206681472 basic_session_run_hooks.py:626] Calling checkpoint listeners after saving checkpoint 0...


KeyboardInterrupt: ignored

In [16]:
# Modify this cell to retrieve model data with the best accuracy
!ls /tmp/run_example/tuner-1/4

4.arch.txt				     model.ckpt-0.meta
4.search_edge.txt			     model.ckpt-5.data-00000-of-00001
checkpoint				     model.ckpt-5.index
eval					     model.ckpt-5.meta
events.out.tfevents.1618606737.e2dde30af08c  replay_config.pbtxt
graph.pbtxt				     saved_model
model.ckpt-0.data-00000-of-00001	     search_generator_0
model.ckpt-0.index


In [17]:
# Show model summary
!cat /tmp/run_example/tuner-1/4/graph.pbtxt

node {
  name: "global_step/Initializer/zeros"
  op: "Const"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@global_step"
      }
    }
  }
  attr {
    key: "_output_shapes"
    value {
      list {
        shape {
        }
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT64
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT64
        tensor_shape {
        }
        int64_val: 0
      }
    }
  }
}
node {
  name: "global_step"
  op: "VarHandleOp"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@global_step"
      }
    }
  }
  attr {
    key: "_output_shapes"
    value {
      list {
        shape {
        }
      }
    }
  }
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT64
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: "globa